In [1]:
import numpy as np
import torch
from models import *
from inception_score import get_inception_score
from torch.utils.data import TensorDataset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Resize, Normalize

/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sharathramku/.local/lib/python3.6/site-packages/tensorflow/python/f

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.


In [2]:
G = torch.load('mnist_logan-b/G_9.pth').to('cuda:1').eval()
E = torch.load('mnist_logan-b/E_9.pth').to('cuda:1').eval()

In [3]:
num_images = 1000

In [4]:
dataset = MNIST(root='.', download=True, transform=Compose([Resize(32), ToTensor(), Normalize((0.5,),(0.5,))]))
dataloader = DataLoader(dataset, batch_size=num_images, shuffle=True)

In [5]:
images1 = np.zeros((num_images, 3, 32, 32))
images2 = np.zeros((num_images, 3, 32, 32))

In [6]:
real = ((next(iter(dataloader))[0]+1)*127).squeeze().numpy()
images1[:,0,:,:] = real.copy()
images1[:,1,:,:] = real.copy()
images1[:,2,:,:] = real.copy()
images1 = images1.astype(np.uint8)

In [7]:
get_inception_score(images1)

Calculating Inception Score with 1000 images in 10 splits
Inception Score calculation time: 34.964546 s


(1.9982408, 0.0710372)

In [8]:
real = next(iter(dataloader))[0]
latents = E(real.to('cuda:1')).detach()
recon = ((G(latents).cpu().detach()+1)*127).squeeze().numpy()
images2[:,0,:,:] = recon.copy()
images2[:,1,:,:] = recon.copy()
images2[:,2,:,:] = recon.copy()
images2 = images2.astype(np.uint8)

In [9]:
get_inception_score(images2)

Calculating Inception Score with 1000 images in 10 splits
Inception Score calculation time: 34.669444 s


(2.0079558, 0.077255085)